<a href="https://colab.research.google.com/github/DawidJag/APP_receipts_settlement/blob/main/rozliczenie_paragonow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
import pandas as pd
import numpy as np



In [118]:
# pip install tinydb

# from tinydb import TinyDB
# from tinydb import Query


# module for very small data bases => finally not used in here

# db = TinyDB('db.txt')           # https://stackoverflow.com/questions/43610854/how-can-i-use-a-text-file-as-database-in-python
# db.insert({'id': 1, 'name': 'car1', 'color': 'green'})
# db.insert({'id': 2, 'name': 'car1', 'color': 'green'})

In [119]:
class Rozliczenie():
  def __init__(self, participants, name):
    self.participants = participants         # list of people
    self.name = name
    self.receipts = []            # adding each receipt  to this list during creation of each receipt

  def get_settle_balance(self):
    settle_balance = {}

    for person in self.participants:
      settle_balance[person] = 0

      for receipt in self.receipts:
        settle_balance[person] += receipt.get_balance_per_person()[person]
    return settle_balance

  def get_split_of_balance(self):           # splitting balance per person for negative and positive
    negative_bal = {}
    positive_bal = {}

    for key, value in self.get_settle_balance().items():
      if value < 0:
        negative_bal[key] = value
      else:
        positive_bal[key] = value

    positive_bal = pd.Series(positive_bal)
    negative_bal = pd.Series(negative_bal)

    positive_bal.sort_values(ascending=True, inplace=True)
    negative_bal.sort_values(ascending=True, inplace=True)
    final_payers = list(negative_bal.index)
    final_receivers = list(positive_bal.index)
    
    return positive_bal, negative_bal, final_payers, final_receivers

# *********************************************************

  # final settlement between all participants
  def final_payments_report(self):
    payments = {}                                                      # dictionary => {'payer1': {'receiver1': amt1, 'receiver2': amt2}}
    positive, negative, payers, receivers = self.get_split_of_balance()

    for payer in payers:
      payments[payer] = {}
      for receiver in receivers:

        if positive[receiver] <= abs(negative[payer]):
          payments[payer].update({receiver: positive[receiver]})       # payment
          negative[payer] += positive[receiver]                        # adding receiver to the dict with payments
          positive[receiver] -= positive[receiver]                     # settle of payment on receiver side
        else:
          payments[payer].update({receiver: abs(negative[payer])})

    return payments
# ********************************************************

class Paragon():
  def __init__(self, participants, payments, private_exp, common_exp_share, category, date, remarks=''):
    self.participants = participants
    self.payments = payments
    self.private_exp = private_exp
    self.common_exp_share = common_exp_share
    self.payers = list(payments.keys())
    self.pay_amt = list(payments.values())
    self.priv_exp_persons = list(private_exp.keys())
    self.priv_exp_amt = list(private_exp.values())
    self.common_exp_persons = list(common_exp_share.keys())
    self.common_exp_amt = sum(self.pay_amt) - sum(self.priv_exp_amt)
    self.category = category
    self.date = date
    self.remarks = remarks
    self.exp_per_person = self.get_exp_per_person()

  def get_common_exp_per_person(self):
    no_of_people = sum(list(self.common_exp_share.values()))
    common_exp_per_person = self.common_exp_amt / no_of_people
    return common_exp_per_person

  def get_exp_per_person(self):
    exp_per_person_dict = {}

    for person in self.participants:
      if self.common_exp_share[person] == 0:
        exp_per_person_dict[person] = self.private_exp[person]
      else:
        exp_per_person_dict[person] = self.private_exp[person] + self.get_common_exp_per_person()
    
    return exp_per_person_dict

  def get_payment_per_person(self):
    return self.payments

  def get_balance_per_person(self):               
    balance = {}
    for person in self.participants:
      if person in self.payers:
        balance[person] = self.get_payment_per_person()[person] - self.get_exp_per_person()[person]
      else:
        balance[person] = -self.get_exp_per_person()[person]
    return balance


In [157]:
# ****************************** Data Base ******************
# created based on:   https://www.techwithtim.net/tutorials/kivy-tutorial/example-gui/
class DataBase:
  def __init__(self, file_name):
    self.filename = file_name
    self.settlements = None
    self.file = None
    self.load()

  def load(self):
    self.file = open(self.filename, "r")
    self.settlements = {}

    for line in self.file:
      settlement, receipt, participants, payments, private_exp, common_exp_share, category, date, remarks = line.strip().split(";")

      if settlement in self.settlements.keys():
        self.settlements[settlement].update({receipt: (participants, payments, private_exp, common_exp_share, category, date, remarks)})
      else:          
        self.settlements[settlement] = {receipt: (participants, payments, private_exp, common_exp_share, category, date, remarks)}
    self.file.close()

  def add_record(self, settlement, receipt, participants, payments, private_exp, common_exp_share, category, date, remarks):
    
    if settlement.strip() in self.settlements.keys():
      if receipt.strip() not in self.settlements[settlement.strip()]:           # checking if specific receipt already exists for that settlement, if not add receipt to the settlement     
        self.settlements[settlement.strip()].update({receipt.strip(): (participants, payments, private_exp, common_exp_share, category, date, remarks)})
        self.save()           
        return 1
      else:
        print("Receipt exists already")
        return -1

    else:
      self.settlements[settlement.strip()] = {receipt.strip(): (participants, payments, private_exp, common_exp_share, category, date, remarks)}
      self.save()
      return 1

  def save(self):
      with open(self.filename, "w") as f:
          for settlement in self.settlements:
            for receipt in self.settlements[settlement]:
              item = self.settlements[settlement][receipt]

              f.write(settlement + ";" + receipt + ";" + item[0] + ";" + item[1] + ";" + item[2] + ";" + item[3] + ";" + item[4] + ";" + item[5] + ";" + item[6] + "\n")
              
  def update_record(self, old_settlement, old_receipt, new_settlement, new_receipt, participants, payments, private_exp, common_exp_share, category, date, remarks):
    self.delete_record(old_settlement, old_receipt)
    self.add_record(new_settlement, new_receipt, participants, payments, private_exp, common_exp_share, category, date, remarks)
    return 1

  def delete_record(self, settlement, receipt):
    self.settlements[settlement].pop(receipt)
    self.save()
    return 1

  def get_settlements_list(self):
    return self.settlements.keys()

  def get_receipts_list(self, settlement):            # for specific settlement
    return self.settlements[settlement].keys()
  


In [158]:
db = DataBase('baza_danych2.txt')

In [166]:
db.settlements

{'rozliczenie1': {'paragon1': ("['Daw','Rob','Aga','Mama']",
   '{"Daw":100,"Rob":150,"Aga":150}',
   '{"Daw":110,"Rob":60,"Aga":100,"Mama":60}',
   '{"Daw":1,"Rob":1,"Aga":1,"Mama":1}',
   'jedzenie',
   '2020-11-22',
   'wyjazd na wakacje')},
 'rozliczenie2': {'paragon2': ('bbb',
   'aaa',
   'aaa',
   'aaa',
   'aaa',
   'aaa',
   'aaa'),
  'paragon3': ('ccc', 'ccc', 'ccc', 'ccc', 'ccc', 'ccc', 'ccc')},
 'rozliczenie3': {'paragon3': ('bbb',
   'aaa',
   'aaa',
   'aaa',
   'aaa',
   'aaa',
   'aaa')}}